# Predicting patient pay for covermymeds

The goal of this notebook is to predict patient pay for claims data. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
import numpy as np


from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor



In [32]:
# fetching data
train_data = pd.read_csv('../train_temp.csv')

In [33]:
print(train_data.shape)
train_data.head()

(10258707, 18)


,Unnamed: 0,tx_date,pharmacy,diagnosis,drug,bin,pcn,group,rejected,patient_pay,drug_phar_bin,drug_phar_grp,drug_phar_pcn,drug_phar,drug_diag,drug_bin,drug_grp,drug_pcn
0,0,2022-05-26,Pharmacy #28,I68.27,branded prazinib,322463,NaN,HO8HUGL,False,11.09,12.120940,11.090000,12.120940,12.702696,12.829066,12.22,11.09,12.22
1,1,2022-01-20,Pharmacy #18,G99.93,branded bovirol,664344,NaN,52H8KH0F83K,False,12.46,14.814396,12.460000,14.814396,13.789535,13.282289,17.61,12.46,17.61
2,2,2022-11-13,Pharmacy #5,I68.27,branded prazinib,664344,TPJD,NaN,False,19.82,13.705529,13.705529,19.820000,12.492624,12.829066,17.11,17.11,19.82
3,3,2022-03-16,Pharmacy #22,I59.87,generic oxasoted,664344,NaN,STGRDKR1J5RD,False,5.94,8.446412,5.940000,8.446412,8.969989,8.423183,6.56,5.94,6.56
4,4,2022-12-17,Pharmacy #24,Q72.66,branded ratin,757349,MSCXSG,DGLGRYP,False,23.83,25.756022,26.652835,26.652835,26.192209,25.035298,23.33,23.40,23.40


In [34]:
list(train_data.columns)

['Unnamed: 0',
 'tx_date',
 'pharmacy',
 'diagnosis',
 'drug',
 'bin',
 'pcn',
 'group',
 'rejected',
 'patient_pay',
 'drug_phar_bin',
 'drug_phar_grp',
 'drug_phar_pcn',
 'drug_phar',
 'drug_diag',
 'drug_bin',
 'drug_grp',
 'drug_pcn']

In [35]:
train_data.drop(["Unnamed: 0"], axis=1, inplace=True)

In [45]:
train_data1=train_data.drop_duplicates()

In [46]:
print(train_data1.shape)
train_data1.head()

(9377718, 17)


,tx_date,pharmacy,diagnosis,drug,bin,pcn,group,rejected,patient_pay,drug_phar_bin,drug_phar_grp,drug_phar_pcn,drug_phar,drug_diag,drug_bin,drug_grp,drug_pcn
0,2022-05-26,Pharmacy #28,I68.27,branded prazinib,322463,NaN,HO8HUGL,False,11.09,12.120940,11.090000,12.120940,12.702696,12.829066,12.22,11.09,12.22
1,2022-01-20,Pharmacy #18,G99.93,branded bovirol,664344,NaN,52H8KH0F83K,False,12.46,14.814396,12.460000,14.814396,13.789535,13.282289,17.61,12.46,17.61
2,2022-11-13,Pharmacy #5,I68.27,branded prazinib,664344,TPJD,NaN,False,19.82,13.705529,13.705529,19.820000,12.492624,12.829066,17.11,17.11,19.82
3,2022-03-16,Pharmacy #22,I59.87,generic oxasoted,664344,NaN,STGRDKR1J5RD,False,5.94,8.446412,5.940000,8.446412,8.969989,8.423183,6.56,5.94,6.56
4,2022-12-17,Pharmacy #24,Q72.66,branded ratin,757349,MSCXSG,DGLGRYP,False,23.83,25.756022,26.652835,26.652835,26.192209,25.035298,23.33,23.40,23.40


In [47]:
for column in train_data1.columns[8:17]:
    try:
        train_data1[column] = np.log10(train_data1[column])
    except (ValueError, AttributeError):
        pass

/Users/dilhumarablat/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [48]:
train_data1.head()

,tx_date,pharmacy,diagnosis,drug,bin,pcn,group,rejected,patient_pay,drug_phar_bin,drug_phar_grp,drug_phar_pcn,drug_phar,drug_diag,drug_bin,drug_grp,drug_pcn
0,2022-05-26,Pharmacy #28,I68.27,branded prazinib,322463,NaN,HO8HUGL,False,1.044932,1.083536,1.044932,1.083536,1.103896,1.108195,1.087071,1.044932,1.087071
1,2022-01-20,Pharmacy #18,G99.93,branded bovirol,664344,NaN,52H8KH0F83K,False,1.095518,1.170684,1.095518,1.170684,1.139550,1.123273,1.245759,1.095518,1.245759
2,2022-11-13,Pharmacy #5,I68.27,branded prazinib,664344,TPJD,NaN,False,1.297104,1.136896,1.136896,1.297104,1.096654,1.108195,1.233250,1.233250,1.297104
3,2022-03-16,Pharmacy #22,I59.87,generic oxasoted,664344,NaN,STGRDKR1J5RD,False,0.773786,0.926672,0.773786,0.926672,0.952792,0.925476,0.816904,0.773786,0.816904
4,2022-12-17,Pharmacy #24,Q72.66,branded ratin,757349,MSCXSG,DGLGRYP,False,1.377124,1.410879,1.425743,1.425743,1.418172,1.398553,1.367915,1.369216,1.369216


In [49]:
kf = KFold(n_splits=5, shuffle=True, random_state=10)
scores =[]; i = 1

cols=['drug_phar_bin', 'drug_phar_grp','drug_phar_pcn', 
    'drug_phar', 'drug_diag', 'drug_bin', 'drug_grp',
    'drug_pcn', 'patient_pay']

for train_index, test_index in kf.split(train_data1):
    train_kf, test_kf = train_data1.iloc[train_index,:], train_data1.iloc[test_index,:]

    models = [ Ridge(alpha=1),
            RandomForestRegressor(max_depth=10)]

    for j in range(len(models)):

        X_temp = train_kf[cols].drop_duplicates()
        X = X_temp[cols[:-1]]
        y = X_temp['patient_pay']

        X_tst = test_kf[cols[:-1]]
        y_tst = test_kf['patient_pay']

        if j==0 :
            models[j].fit(np.log(X),np.log(y))
            test_predict = np.exp(models[j].predict(np.log(X_tst)))
            
        else :
            models[j].fit(X,y)
            test_predict = models[j].predict(X_tst)

        r2 = r2_score(test_predict,y_tst)
        mean_abs = mean_absolute_error(test_predict,y_tst)
        mean_sqr = mean_squared_error(test_predict,y_tst)
        mean_perc = mean_absolute_percentage_error(test_predict,y_tst)

        scores.append((models[j],i,r2,mean_abs,mean_sqr,mean_perc*100))
    i+=1

kf_df = pd.DataFrame(scores,columns = ['model','fold_idx', 'r2_score' ,'mean_absolute' ,'mean_sqr' , 'mean_abs_percent'])

for i in range(len(kf_df)):
    if i%2 == 0:
        kf_df.iloc[i,0] = 'Ridge_truncated'
    else:
        kf_df.iloc[i,0] = 'Random Forest'
kf_df

,model,fold_idx,r2_score,mean_absolute,mean_sqr,mean_abs_percent
0,Ridge_truncated,1,0.980969,0.031119,0.002489,2.612724
1,Random Forest,1,0.991967,0.013807,0.001081,0.975912
2,Ridge_truncated,2,0.980882,0.031161,0.002496,2.616064
3,Random Forest,2,0.991904,0.013842,0.001088,0.979058
4,Ridge_truncated,3,0.980993,0.031139,0.002490,2.613242
5,Random Forest,3,0.991957,0.013790,0.001085,0.974289
6,Ridge_truncated,4,0.980894,0.031185,0.002502,2.617255
7,Random Forest,4,0.991935,0.013835,0.001087,0.978478
8,Ridge_truncated,5,0.980947,0.031125,0.002488,2.613013
9,Random Forest,5,0.991900,0.013899,0.001089,0.983610


# Tree Based Models

Here are going to use two tree based models :

1) Random Forests Regressor

2) Gradient Boost Regressor

Since tree based models take longer time to train and we have over $10$ million data, we are going to truncate duplicate variables when training our tree models.

### Random Forests and Ridge regression on the truncated data

In [72]:
kf = KFold(n_splits=5, shuffle=True, random_state=10)
scores =[]; i = 1

cols=['drug_phar_bin', 'drug_phar_grp','drug_phar_pcn', 
    'drug_phar', 'drug_diag', 'drug_bin', 'drug_grp',
    'drug_pcn', 'patient_pay']

for train_index, test_index in kf.split(train_data1):
    train_kf, test_kf = train_data1.iloc[train_index,:], train_data1.iloc[test_index,:]

    models = [ Ridge(alpha=1),
            RandomForestRegressor(max_depth=10)]

    for j in range(len(models)):

        X_temp = train_kf[cols].drop_duplicates()
        X = X_temp[cols[:-1]]
        y = X_temp['patient_pay']

        X_tst = test_kf[cols[:-1]]
        y_tst = test_kf['patient_pay']

        if j==0 :
            models[j].fit(np.log(X),np.log(y))
            test_predict = np.exp(models[j].predict(np.log(X_tst)))
            
        else :
            models[j].fit(X,y)
            test_predict = models[j].predict(X_tst)

        r2 = r2_score(test_predict,y_tst)
        mean_abs = mean_absolute_error(test_predict,y_tst)
        mean_sqr = mean_squared_error(test_predict,y_tst)
        mean_perc = mean_absolute_percentage_error(test_predict,y_tst)

        scores.append((models[j],i,r2,mean_abs,mean_sqr,mean_perc*100))
    i+=1

kf_df = pd.DataFrame(scores,columns = ['model','fold_idx', 'r2_score' ,'mean_absolute' ,'mean_sqr' , 'mean_abs_percent'])

for i in range(len(kf_df)):
    if i%2 == 0:
        kf_df.iloc[i,0] = 'Ridge_truncated'
    else:
        kf_df.iloc[i,0] = 'Random Forest'
kf_df

,model,fold_idx,r2_score,mean_absolute,mean_sqr,mean_abs_percent
0,Ridge_truncated,1,0.980969,0.031119,0.002489,2.612724
1,Random Forest,1,0.991979,0.013751,0.001080,0.971666
2,Ridge_truncated,2,0.980882,0.031161,0.002496,2.616064
3,Random Forest,2,0.991908,0.013813,0.001088,0.976874
4,Ridge_truncated,3,0.980993,0.031139,0.002490,2.613242
5,Random Forest,3,0.991985,0.013768,0.001081,0.972986
6,Ridge_truncated,4,0.980894,0.031185,0.002502,2.617255
7,Random Forest,4,0.991919,0.013883,0.001089,0.982163
8,Ridge_truncated,5,0.980947,0.031125,0.002488,2.613013
9,Random Forest,5,0.991932,0.013771,0.001084,0.973497


In [74]:
kf_df

,model,fold_idx,r2_score,mean_absolute,mean_sqr,mean_abs_percent
0,Ridge_truncated,1,0.980969,1.031608,1.002492,13.636144
1,Random Forest,1,0.991979,1.013846,1.001080,2.642342
2,Ridge_truncated,2,0.980882,1.031651,1.002499,13.681765
3,Random Forest,2,0.991908,1.013908,1.001088,2.656139
4,Ridge_truncated,3,0.980993,1.031629,1.002493,13.643216
5,Random Forest,3,0.991985,1.013863,1.001082,2.645834
6,Ridge_truncated,4,0.980894,1.031676,1.002505,13.698064
7,Random Forest,4,0.991919,1.013980,1.001090,2.670226
8,Ridge_truncated,5,0.980947,1.031615,1.002491,13.640084
9,Random Forest,5,0.991932,1.013866,1.001085,2.647185


In [58]:
test_data = pd.read_csv('../test_temp.csv')

In [59]:
print(test_data.shape)
test_data.head()

(2564677, 34)


,Unnamed: 0,tx_date,pharmacy,diagnosis,drug,bin,pcn,group,rejected,patient_pay,...,drug_grp_y,drug_pcn_y,drug_phar_bin,drug_phar_grp,drug_phar_pcn,drug_phar,drug_diag,drug_bin,drug_grp,drug_pcn
0,0,2022-03-16,Pharmacy #22,H60.83,generic colifunene,571569,KB38N,6BYJBW,False,11.43,...,11.43,11.43,12.090000,11.43000,11.430000,13.525341,13.559247,11.43,11.43,11.43
1,1,2022-12-10,Pharmacy #29,I68.27,branded prazinib,571569,KB38N,6BYJBW,False,5.79,...,5.79,5.79,5.931600,5.79000,5.790000,12.766380,12.829066,5.79,5.79,5.79
2,2,2022-09-25,Pharmacy #27,G51.87,generic prazinib,664344,NaN,TFZOR5R49,False,10.72,...,10.72,NaN,10.592317,10.72000,10.592317,10.658195,10.674657,10.72,10.72,10.72
3,3,2022-09-20,Pharmacy #40,U61.13,branded momudobatin,96934,S76J7V6,NaN,False,62.35,...,NaN,63.18,58.997260,58.99726,62.512836,57.857892,53.704793,62.98,62.98,63.18
4,4,2022-01-19,Pharmacy #14,Z98.86,generic keglusited,691847,XH4T3,GOM8K0,False,15.55,...,15.55,15.55,15.682487,15.55000,15.550000,13.673011,13.503777,17.91,15.55,15.55


In [60]:
test_data.drop(["Unnamed: 0"], axis=1, inplace=True)

In [61]:
test_data1=test_data.drop_duplicates()

In [62]:
print(test_data.shape)
test_data.head()

(2564677, 33)


,tx_date,pharmacy,diagnosis,drug,bin,pcn,group,rejected,patient_pay,drug_phar_bin_x,...,drug_grp_y,drug_pcn_y,drug_phar_bin,drug_phar_grp,drug_phar_pcn,drug_phar,drug_diag,drug_bin,drug_grp,drug_pcn
0,2022-03-16,Pharmacy #22,H60.83,generic colifunene,571569,KB38N,6BYJBW,False,11.43,12.090000,...,11.43,11.43,12.090000,11.43000,11.430000,13.525341,13.559247,11.43,11.43,11.43
1,2022-12-10,Pharmacy #29,I68.27,branded prazinib,571569,KB38N,6BYJBW,False,5.79,5.931600,...,5.79,5.79,5.931600,5.79000,5.790000,12.766380,12.829066,5.79,5.79,5.79
2,2022-09-25,Pharmacy #27,G51.87,generic prazinib,664344,NaN,TFZOR5R49,False,10.72,10.592317,...,10.72,NaN,10.592317,10.72000,10.592317,10.658195,10.674657,10.72,10.72,10.72
3,2022-09-20,Pharmacy #40,U61.13,branded momudobatin,96934,S76J7V6,NaN,False,62.35,58.997260,...,NaN,63.18,58.997260,58.99726,62.512836,57.857892,53.704793,62.98,62.98,63.18
4,2022-01-19,Pharmacy #14,Z98.86,generic keglusited,691847,XH4T3,GOM8K0,False,15.55,15.682487,...,15.55,15.55,15.682487,15.55000,15.550000,13.673011,13.503777,17.91,15.55,15.55


In [63]:
for column in test_data.columns[8:33]:
    try:
        test_data[column] = np.log10(test_data[column])
    except (ValueError, AttributeError):
        pass

In [64]:
test_data.head()

,tx_date,pharmacy,diagnosis,drug,bin,pcn,group,rejected,patient_pay,drug_phar_bin_x,...,drug_grp_y,drug_pcn_y,drug_phar_bin,drug_phar_grp,drug_phar_pcn,drug_phar,drug_diag,drug_bin,drug_grp,drug_pcn
0,2022-03-16,Pharmacy #22,H60.83,generic colifunene,571569,KB38N,6BYJBW,False,1.058046,1.082426,...,1.058046,1.058046,1.082426,1.058046,1.058046,1.131148,1.132236,1.058046,1.058046,1.058046
1,2022-12-10,Pharmacy #29,I68.27,branded prazinib,571569,KB38N,6BYJBW,False,0.762679,0.773172,...,0.762679,0.762679,0.773172,0.762679,0.762679,1.106068,1.108195,0.762679,0.762679,0.762679
2,2022-09-25,Pharmacy #27,G51.87,generic prazinib,664344,NaN,TFZOR5R49,False,1.030195,1.024991,...,1.030195,NaN,1.024991,1.030195,1.024991,1.027684,1.028354,1.030195,1.030195,1.030195
3,2022-09-20,Pharmacy #40,U61.13,branded momudobatin,96934,S76J7V6,NaN,False,1.794836,1.770832,...,NaN,1.800580,1.770832,1.770832,1.795969,1.762363,1.730013,1.799203,1.799203,1.800580
4,2022-01-19,Pharmacy #14,Z98.86,generic keglusited,691847,XH4T3,GOM8K0,False,1.191730,1.195415,...,1.191730,1.191730,1.195415,1.191730,1.191730,1.135864,1.130455,1.253096,1.191730,1.191730


In [75]:
cols = ['drug_phar_bin','drug_phar_grp','drug_phar_pcn','drug_diag','drug_bin',
 'drug_grp','drug_pcn','patient_pay']

train_data2=train_data1[cols].drop_duplicates()

rf = RandomForestRegressor(max_depth=10)

rf.fit(train_data2[cols[:-1]], train_data2['patient_pay'])



RandomForestRegressor(max_depth=10)

In [104]:
pred = rf.predict(test_data[cols[:-1]])
y_test = test_data['patient_pay']

print('Test Predictions :')

print('R2 score : ' , r2_score(pred,y_test))
print('MAE : ' ,mean_absolute_error(pred,y_test))
print('RMSE : ' ,np.sqrt(mean_squared_error(pred,y_test)))
print('MAPE : ',mean_absolute_percentage_error(pred,y_test)*100)

Test Predictions :
R2 score :  0.9908779497813666
MAE :  0.014747200772304932
RMSE :  0.03470049612862834
MAPE :  1.042234729767917


In [105]:
rf.feature_importances_

array([3.34108043e-04, 7.55138094e-01, 1.52833848e-02, 3.54111353e-04,
       1.29812523e-03, 1.20111971e-01, 1.07480205e-01])

In [142]:
test = pd.read_csv('../test_data.csv')

In [143]:
test['predictions'] = pred

In [144]:
test['patient_pay']= np.log10(test['patient_pay'])

In [145]:
test

,Unnamed: 0,tx_date,pharmacy,diagnosis,drug,bin,pcn,group,rejected,patient_pay,predictions
0,2526655,2022-03-16,Pharmacy #22,H60.83,generic colifunene,571569,KB38N,6BYJBW,False,1.058046,1.062199
1,13162765,2022-12-10,Pharmacy #29,I68.27,branded prazinib,571569,KB38N,6BYJBW,False,0.762679,0.762679
2,10002547,2022-09-25,Pharmacy #27,G51.87,generic prazinib,664344,NaN,TFZOR5R49,False,1.030195,1.030855
3,9777107,2022-09-20,Pharmacy #40,U61.13,branded momudobatin,96934,S76J7V6,NaN,False,1.794836,1.804221
4,603661,2022-01-19,Pharmacy #14,Z98.86,generic keglusited,691847,XH4T3,GOM8K0,False,1.191730,1.192098
...,...,...,...,...,...,...,...,...,...,...,...
2564672,12367815,2022-11-20,Pharmacy #33,H60.83,generic colifunene,691847,NaN,I4UYEP84W3,False,1.191730,1.192098
2564673,346542,2022-01-12,Pharmacy #9,Q72.66,branded ratin,725700,1UQC,NaN,False,1.570660,1.325022
2564674,2116353,2022-03-05,Pharmacy #11,W50.87,generic cupitelol,664344,NaN,52H8KH0F83K,False,0.785330,0.785392
2564675,9218879,2022-09-08,Pharmacy #0,G51.87,branded prazinib,322463,T52GV,EVD4X5,False,1.027757,1.025621


In [150]:
test[ abs(test.predictions - test.patient_pay) > 0.2]

,Unnamed: 0,tx_date,pharmacy,diagnosis,drug,bin,pcn,group,rejected,patient_pay,predictions
391,551927,2022-01-18,Pharmacy #22,U61.13,generic momudobatin,96934,S76J7V6,NaN,False,1.611192,1.376145
608,583928,2022-01-18,Pharmacy #29,P77.13,generic cicrochoric,725700,9C5MOR3,S2QKZ0OFNWS6X,False,1.397245,1.152591
1181,314319,2022-01-12,Pharmacy #2,M42.24,branded tocilic,757349,MSCXSG,DGLGRYP,False,1.732956,1.524618
1396,2613815,2022-03-18,Pharmacy #30,C98.15,branded ratin,664344,CS8580,NaN,False,1.659155,1.438904
1563,10816141,2022-10-15,Pharmacy #37,Q72.66,generic momudobatin,691847,NC7EN,NaN,False,1.616476,1.322792
...,...,...,...,...,...,...,...,...,...,...,...
2562991,3906854,2022-04-22,Pharmacy #42,U06.52,branded ratin,725700,1UQC,NaN,False,1.570660,1.360835
2563228,125022,2022-01-07,Pharmacy #10,H36.57,branded antimab,691847,NC7EN,NaN,False,1.546049,1.261705
2564188,177150,2022-01-08,Pharmacy #33,M06.85,branded isobrovelin,96934,S76J7V6,NaN,False,1.732233,1.474800
2564651,3007793,2022-03-27,Pharmacy #41,Q72.66,generic ratin,691847,NC7EN,NaN,False,1.498448,1.286905


In [147]:
test[(test.drug=='branded tugesutin') &(test.pcn=='CS8580') &(test.pharmacy=='Pharmacy #29')]

,Unnamed: 0,tx_date,pharmacy,diagnosis,drug,bin,pcn,group,rejected,patient_pay,predictions
35869,3039484,2022-03-28,Pharmacy #29,U06.52,branded tugesutin,664344,CS8580,NaN,False,2.386980,2.267438
341765,5277794,2022-05-28,Pharmacy #29,U61.13,branded tugesutin,664344,CS8580,NaN,False,2.267406,2.267438
430536,8700546,2022-08-24,Pharmacy #29,U06.52,branded tugesutin,664344,CS8580,NaN,False,2.267406,2.267438
521500,7376180,2022-07-22,Pharmacy #29,Q72.66,branded tugesutin,664344,CS8580,NaN,False,2.267406,2.267438
826478,11132998,2022-10-22,Pharmacy #29,U61.13,branded tugesutin,664344,CS8580,NaN,False,2.267406,2.267438
1939276,1925762,2022-02-25,Pharmacy #29,Q72.66,branded tugesutin,664344,CS8580,NaN,False,2.267406,2.267438
2460784,5941564,2022-06-16,Pharmacy #29,Q72.66,branded tugesutin,664344,CS8580,NaN,False,2.267406,2.267438


In [148]:
test[(test.drug=='branded prefluflomycin') &(test.pcn=='MSCXSG') &(test.pharmacy=='Pharmacy #37')]

,Unnamed: 0,tx_date,pharmacy,diagnosis,drug,bin,pcn,group,rejected,patient_pay,predictions
54846,4537468,2022-05-10,Pharmacy #37,C14.36,branded prefluflomycin,757349,MSCXSG,DGLGRYP,False,2.429639,2.432565
942569,12811200,2022-12-03,Pharmacy #37,C14.36,branded prefluflomycin,757349,MSCXSG,DGLGRYP,False,2.429639,2.432565
944951,7337982,2022-07-21,Pharmacy #37,I91.91,branded prefluflomycin,757349,MSCXSG,DGLGRYP,False,2.429639,2.432565
2213401,8551379,2022-08-21,Pharmacy #37,I91.91,branded prefluflomycin,757349,MSCXSG,DGLGRYP,False,2.429639,2.432565
2507119,166551,2022-01-08,Pharmacy #37,I91.91,branded prefluflomycin,757349,MSCXSG,DGLGRYP,False,2.551645,2.432565
